In [ ]:
#LIMITATIONS:
#    Unable to filter by ratings (no input parameter available)

In [121]:
# Import required dependencies
import json
import time
import csv
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from config import api_key, client_id

In [122]:
#Prepare empty lists to store all data
restaurant_name = []
restaurant_id = []
restaurant_rating = []
restaurant_reviewcount = []
restaurant_city = []
restaurant_state = []
restaurant_zipcode = []
restaurant_price=[]
restaurant_type = []
restaurant_category = []

#Necessary to pass header in each API request.  Do not change this.
headers = {'Authorization': f"Bearer {api_key}"}
base_url = 'https://api.yelp.com/v3/businesses/search?'
base_filters = '&categories=Restaurants&limit=50&is_closed=0'

# Construct the URL to request Yelp API data

url_location = ['San Francisco, CA', 'New York, NY', 'Des Moines, Iowa', 'Omaha, Nebraska']
url_price = [1, 2, 3, 4]
url_offset = np.arange(0, 850, 50)

In [123]:
#Yelp provides us with a list of 50 results per query

#loop through different page sets (e.g. 0-50, 51-100, 101-150...)
for off in url_offset:

    #loop through list of rural and suburb locations
    for location in url_location:

        #loop through list of prices $-$$$$
        for price in url_price:

            #prepare query URL for each price range and each zip code region
            query_url = f'{base_url}&location={location}&price={price}{base_filters}&offset={off}'

            #extract data into a response variable
            response = requests.get(query_url, headers=headers).json()

            restaurant_list = response['businesses']

            #loop through responses and append data to empty lists
            for restaurant in restaurant_list:
                restaurant_name.append(restaurant["name"])
                restaurant_id.append(restaurant["id"])
                restaurant_rating.append(restaurant["rating"])
                restaurant_reviewcount.append(restaurant["review_count"])
                restaurant_price.append(len(restaurant["price"]))
                restaurant_city.append(restaurant["location"]["city"])
                restaurant_state.append(restaurant["location"]["state"])
                restaurant_zipcode.append(restaurant["location"]["zip_code"])
                restaurant_type.append(restaurant["categories"][0]["title"])

            #prepare dataframe from dictionary
            restaurant_df = pd.DataFrame({'Restaurant':restaurant_name, 
                               'ID':restaurant_id,
                               'Rating':restaurant_rating, 
                               'Reviews': restaurant_reviewcount, 
                               'Price':restaurant_price, 
                               'City': restaurant_city,
                               'State': restaurant_state,
                               'ZipCode':restaurant_zipcode,
                               #need to be able to derive multiple categories
                               'Category':restaurant_type})

In [125]:
restaurant_df = restaurant_df.drop_duplicates(subset=['Restaurant', 'City'], keep=False)

In [126]:
#drop duplicate records by restaurant and city
restaurant_df.dropna(how='any')
restaurant_df.count()

Restaurant    6281
ID            6281
Rating        6281
Reviews       6281
Price         6281
City          6281
State         6281
ZipCode       6281
Category      6281
dtype: int64

In [127]:
#investigate data by outputting results to CSV file
restaurant_df.to_csv("Restaurant_Info3.csv", index=False, encoding='utf-8')

In [135]:
restaurant_df['City'].value_counts()

New York              1969
San Francisco         1882
Omaha                  757
Brooklyn               441
Des Moines             374
West Des Moines        141
Ankeny                  97
Astoria                 83
Urbandale               58
Clive                   58
Bellevue                45
Altoona                 36
La Vista                36
Johnston                33
Papillion               32
Jersey City             21
Long Island City        21
Grimes                  18
Staten Island           16
Jackson Heights         13
Windsor Heights         13
Elmhurst                13
Hoboken                 10
Woodside                 9
Ralston                  8
Carlisle                 7
Norwalk                  6
Sunnyside                5
Manhattan                5
Corona                   5
                      ... 
Council Bluffs           3
Williamsburg             2
Cumming                  2
East Elmhurst            2
Bedford-Stuyvesant       2
W. Des Moines            2
M

In [136]:
restaurant_df.describe()

,Rating,Reviews,Price
count,6281.000000,6281.000000,6281.000000
mean,3.913310,447.915937,1.930107
std,0.549181,648.403993,0.888561
min,1.000000,1.000000,1.000000
25%,3.500000,62.000000,1.000000
50%,4.000000,243.000000,2.000000
75%,4.000000,578.000000,2.000000
max,5.000000,9848.000000,4.000000


In [137]:
#examples of getting data by groups
sanfrancisco = restaurant_df.groupby('City').get_group('San Francisco')
sanfrancisco.sort_values("Rating")

,Restaurant,ID,Rating,Reviews,Price,City,State,ZipCode,Category
814,Mandarin Lounge,mQcPH4b77-fPh7yFuqLctQ,1.0,1,4,San Francisco,CA,94104,Lounges
2992,Sundaze,LwC-tKCgWQzE0iRpAaLdKw,1.5,4,3,San Francisco,CA,94103,Lounges
2988,Outtakes Cafe,jWd3cVgcueS_FHsUUDF6aw,1.5,8,3,San Francisco,CA,94118,Coffee & Tea
2993,Bernal Supperclub,ySRDLQfvybSdOtOUMZeNiQ,1.5,2,3,San Francisco,CA,94110,American (New)
2991,Café Kendrick,uhkiEDq94-dA8_XgGfXN4Q,2.0,2,3,San Francisco,CA,94118,Bagels
2985,50 on Eight Restaurant,mSkgRBiutwvOit8Axx5cRA,2.0,7,3,San Francisco,CA,94103,American (Traditional)
2972,Tabu Lounge,y0j4nhd8rYAyZgdsHOXjPQ,2.0,48,3,San Francisco,CA,94107,Dance Clubs
2968,Hotel Whitcomb Restaurant,Lce1OEkv52uBe3ZnI9quOg,2.5,34,3,San Francisco,CA,94103,Bars
2969,Fat City,pEQeTXqW3Zj94hiQqvcKjA,2.5,34,3,San Francisco,CA,94103,Bars
2444,Harvest Urban Market,Lyl7fmCru0IAk0vl96KPfg,2.5,364,3,San Francisco,CA,94103,Specialty Food
